# 使用 TensorFlow 做卷积

让我们用所学知识在 TensorFlow 里构建真的 CNNs。在下面的练习中，你需要设定卷积核滤波器（filters）的维度，weight，bias。这在很大程度上来说是 TensorFlow CNNs 最难的部分。一旦你知道如何设置这些属性的大小，应用 CNNs 会很方便。


## 回顾

你应该看一下[二维卷积的文档](https://www.tensorflow.org/api_guides/python/nn#Convolution)。文档大部分都很清楚，`padding`这一部分，可能会有点难以理解。`padding` 会根据你给出的 `'VALID'` 或者 `'SAME'` 参数，做相应改变。

这些也是需要你回顾的：

- TensorFlow 变量。
- Truncated 正态分布 - 在 TensorFlow 中你需要在一个正态分布的区间中初始化你的权值。
- 根据输入大小、滤波器大小，来决定输出维度（如下所示）。你用这个来决定滤波器应该是什么样：

```
 new_height = (input_height - filter_height + 2 * P)/S + 1
 new_width = (input_width - filter_width + 2 * P)/S + 1
```

## 说明

- 在`conv2d`函数中完成所有 `TODO`。
- 设定 `strides` 、 `padding` 、 `filter weight/bias (F_w and F_b)` 输出是 `(1, 2, 2, 3)`。除了 `strides` 所有变量的类型都应该为 TensorFlow 变量。


In [ ]:
"""
Setup the strides, padding and filter weight/bias such that
the output shape is (1, 2, 2, 3).
"""
import tensorflow as tf
import numpy as np

# `tf.nn.conv2d` requires the input be 4D (batch_size, height, width, depth)
# (1, 4, 4, 1)
x = np.array([
    [0, 1, 0.5, 10],
    [2, 2.5, 1, -8],
    [4, 0, 5, 6],
    [15, 1, 2, 3]], dtype=np.float32).reshape((1, 4, 4, 1))
X = tf.constant(x)


def conv2d(input):
    # Filter (weights and bias)
    # The shape of the filter weight is (height, width, input_depth, output_depth)
    # The shape of the filter bias is (output_depth,)
    # TODO: Define the filter weights `F_W` and filter bias `F_b`.
    # NOTE: Remember to wrap them in `tf.Variable`, they are trainable parameters after all.
    F_W = tf.Variable(tf.truncated_normal([2,2,1,3]))
    F_b = tf.Variable(tf.zeros(3))
    # TODO: Set the stride for each dimension (batch_size, height, width, depth)
    strides = [1, 2, 2, 1]
    # TODO: set the padding, either 'VALID' or 'SAME'.
    padding = "SAME"
    # https://www.tensorflow.org/versions/r0.11/api_docs/python/nn.html#conv2d
    # `tf.nn.conv2d` does not include the bias computation so we have to add it ourselves after.
    return tf.nn.conv2d(input, F_W, strides, padding) + F_b

out = conv2d(X)


# 解决方案

这是我的做法。** 注意 ** ：有不止一种方法得到正确的输出维度，你的答案可能会跟我的有所不同。

```
def conv2d(input):
    # Filter (weights and bias)
    F_W = tf.Variable(tf.truncated_normal((2, 2, 1, 3)))
    F_b = tf.Variable(tf.zeros(3))
    strides = [1, 2, 2, 1]
    padding = 'VALID'
    return tf.nn.conv2d(input, F_W, strides, padding) + F_b
```

我想要把输入的 `(1, 4, 4, 1)` 转变成 `(1, 2, 2, 3)`。padding 方法我选择 `'VALID'`。我觉得这更容易理解，也得到了我想要的结果。

```
out_height = ceil(float(in_height - filter_height + 1) / float(strides[1]))
out_width  = ceil(float(in_width - filter_width + 1) / float(strides[2]))
```

把值带入

```
out_height = ceil(float(4 - 2 + 1) / float(2)) = ceil(1.5) = 2
out_width  = ceil(float(4 - 2 + 1) / float(2)) = ceil(1.5) = 2
```

要把深度从 1 变成 3。我要把我滤波器的输出做相应的设置：

```
F_W = tf.Variable(tf.truncated_normal((2, 2, 1, 3))) # (height, width, input_depth, output_depth)
F_b = tf.Variable(tf.zeros(3)) # (output_depth)
```

输入的深度是 1，所以我选择 1 作为滤波器的 `input_depth`。

